In [3]:
# going to the root

import os
os.chdir("../")
os.getcwd()

'/home/izam/coding/PDF-chatbot'

In [11]:
# import some essential guys

from PyPDF2 import PdfReader
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
import os
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from dotenv import load_dotenv

In [12]:
# loading the api key

load_dotenv()
key = os.environ.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = key

In [13]:
# loading the pdf

pdf_reader = PdfReader("data/coding.pdf")

In [17]:
# getting the text from pdf

full_text = ""
for i, page in enumerate(pdf_reader.pages):
    text = page.extract_text()
    if text:
        full_text += text

In [18]:
# lemme check the text

print(f"length of the text: {len(full_text)}")
full_text[:10]

length of the text: 1136235


'Essential '

In [39]:
# chunking the document

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 512,
    chunk_overlap = 32,
    length_function = len,
)

texts = text_splitter.split_text(full_text)

In [40]:
len(texts)

1160

In [41]:
# download the embeddings from OpenAI

embeddings = OpenAIEmbeddings()

In [42]:
# convert the chunks to vectors

docsearch = FAISS.from_texts(texts[:300], embeddings)

Retrying langchain_community.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-embedding-ada-002 in organization org-kYjBNIE5PXunOKtDmOa9LEAK on tokens per min (TPM): Limit 150000, Used 53406, Requested 102493. Please try again in 2.359s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..


In [43]:
# now we can query it

chain = load_qa_chain(OpenAI(), chain_type="stuff")

In [45]:
query = "how can i learn coding ?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' There are many ways to learn coding, but here are a few suggestions:\n\n1. Take an online course: There are many websites that offer online coding courses, such as Codeacademy, Udemy, and Coursera. These courses often have structured lessons and assignments, making it easy to learn at your own pace.\n\n2. Join a coding community: There are many online communities and forums where coders share their knowledge and help each other out. Joining one of these communities can give you access to a wealth of resources and support.\n\n3. Attend a coding bootcamp: Coding bootcamps are intensive, in-person programs that teach coding skills in a short amount of time. They are a great option for those looking to jumpstart their coding career.\n\n4. Practice, practice, practice: Coding is a skill that requires practice to master. Set aside time each day to work on coding projects, and try to challenge yourself with new and more difficult projects as you improve.\n\nRemember, learning to code takes 